<a href="https://colab.research.google.com/github/ameympatil/MIT-ML/blob/main/Keras_tuner_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X = df.drop('Outcome',axis=1)
y = df['Outcome']

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
X.shape

(768, 8)

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [58]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [9]:
model = Sequential()

model.add(Dense(32,input_dim=X_train.shape[1],activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [10]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=32)

Epoch 1/20
20/20 [==============================] - 3s 13ms/step - loss: 0.7761 - accuracy: 0.4756 - val_loss: 0.7162 - val_accuracy: 0.5325
Epoch 2/20
20/20 [==============================] - 0s 5ms/step - loss: 0.6812 - accuracy: 0.6075 - val_loss: 0.6466 - val_accuracy: 0.7013
Epoch 3/20
20/20 [==============================] - 0s 5ms/step - loss: 0.6220 - accuracy: 0.6710 - val_loss: 0.5953 - val_accuracy: 0.7662
Epoch 4/20
20/20 [==============================] - 0s 5ms/step - loss: 0.5807 - accuracy: 0.6954 - val_loss: 0.5639 - val_accuracy: 0.7792
Epoch 5/20
20/20 [==============================] - 0s 5ms/step - loss: 0.5557 - accuracy: 0.7329 - val_loss: 0.5425 - val_accuracy: 0.8052
Epoch 6/20
20/20 [==============================] - 0s 5ms/step - loss: 0.5383 - accuracy: 0.7476 - val_loss: 0.5268 - val_accuracy: 0.7987
Epoch 7/20
20/20 [==============================] - 0s 5ms/step - loss: 0.5246 - accuracy: 0.7524 - val_loss: 0.5146 - val_accuracy: 0.7922
Epoch 8/20
20/20 [=

In [11]:
! pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.8 MB/s 
     |████████████████████████████████| 1.6 MB 52.1 MB/s 


In [12]:
# Tuning Optimizer
import keras_tuner as kt

In [13]:
def build_model(hp):
  model = Sequential()
  
  model.add(Dense(32,input_dim=8,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  opt = hp.Choice('opt',values=['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [14]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='KT',
    project_name='Optimizers'
)

In [15]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.5649350881576538

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 07s


In [16]:
tuner.get_best_hyperparameters()[0].values

{'opt': 'adam'}

In [17]:
tuner.results_summary()

Results summary
Results in KT/Optimizers
Showing 10 best trials
Trial summary
Hyperparameters:
opt: adam
Score: 0.798701286315918
Trial summary
Hyperparameters:
opt: rmsprop
Score: 0.7662337422370911
Trial summary
Hyperparameters:
opt: sgd
Score: 0.6948052048683167
Trial summary
Hyperparameters:
opt: adadelta
Score: 0.5649350881576538


In [18]:
model = tuner.get_best_models(num_models=1)[0]

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=32,epochs=50,initial_epoch=6)

Epoch 7/50
20/20 [==============================] - 1s 22ms/step - loss: 0.5307 - accuracy: 0.7459 - val_loss: 0.5111 - val_accuracy: 0.8052
Epoch 8/50
20/20 [==============================] - 0s 7ms/step - loss: 0.5164 - accuracy: 0.7524 - val_loss: 0.5003 - val_accuracy: 0.7922
Epoch 9/50
20/20 [==============================] - 0s 5ms/step - loss: 0.5047 - accuracy: 0.7590 - val_loss: 0.4918 - val_accuracy: 0.7987
Epoch 10/50
20/20 [==============================] - 0s 7ms/step - loss: 0.4954 - accuracy: 0.7638 - val_loss: 0.4850 - val_accuracy: 0.7922
Epoch 11/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4875 - accuracy: 0.7638 - val_loss: 0.4777 - val_accuracy: 0.8052
Epoch 12/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4803 - accuracy: 0.7687 - val_loss: 0.4737 - val_accuracy: 0.7987
Epoch 13/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4753 - accuracy: 0.7720 - val_loss: 0.4704 - val_accuracy: 0.7987
Epoch 14/50
20/

In [21]:
# tuning no. of neurons

def build_model(hp):
  model = Sequential()

  units = hp.Int('values',8,128)

  model.add(Dense(units=units,input_dim=8,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [22]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='KT',
    project_name='Neurons'
)

In [23]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 11s


In [24]:
tuner.results_summary()

Results summary
Results in KT/Neurons
Showing 10 best trials
Trial summary
Hyperparameters:
values: 102
Score: 0.798701286315918
Trial summary
Hyperparameters:
values: 45
Score: 0.798701286315918
Trial summary
Hyperparameters:
values: 27
Score: 0.7857142686843872
Trial summary
Hyperparameters:
values: 118
Score: 0.7792207598686218
Trial summary
Hyperparameters:
values: 73
Score: 0.7597402334213257


In [25]:
tuner.get_best_hyperparameters()[0].values

{'values': 102}

In [26]:
model = tuner.get_best_models(num_models=1)[0]

In [27]:
model.fit(X_train,y_train,epochs=50,batch_size=32,validation_data=(X_test,y_test),initial_epoch=5)

Epoch 6/50
20/20 [==============================] - 1s 12ms/step - loss: 0.5131 - accuracy: 0.7687 - val_loss: 0.4916 - val_accuracy: 0.7922
Epoch 7/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4909 - accuracy: 0.7801 - val_loss: 0.4776 - val_accuracy: 0.7922
Epoch 8/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4777 - accuracy: 0.7801 - val_loss: 0.4698 - val_accuracy: 0.7792
Epoch 9/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4681 - accuracy: 0.7834 - val_loss: 0.4674 - val_accuracy: 0.7792
Epoch 10/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4622 - accuracy: 0.7801 - val_loss: 0.4644 - val_accuracy: 0.7922
Epoch 11/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4575 - accuracy: 0.7769 - val_loss: 0.4640 - val_accuracy: 0.7987
Epoch 12/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4549 - accuracy: 0.7785 - val_loss: 0.4607 - val_accuracy: 0.7922
Epoch 13/50
20/2

In [28]:
## Tuning Layers

def build_model(hp):
  model = Sequential()

  model.add(Dense(102,activation='relu',input_dim=8))
  
  for i in range(hp.Int('num',1,10)):
    model.add(Dense(102,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [29]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    directory='KT',
    project_name='Layers'
)

In [30]:
tuner.search(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Trial 7 Complete [00h 00m 04s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 24s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'num': 7}

In [32]:
model = tuner.get_best_models(num_models=1)[0]

In [34]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=32,initial_epoch=5)

Epoch 6/50
20/20 [==============================] - 0s 8ms/step - loss: 0.4217 - accuracy: 0.8094 - val_loss: 0.4942 - val_accuracy: 0.7597
Epoch 7/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4035 - accuracy: 0.8192 - val_loss: 0.6786 - val_accuracy: 0.7532
Epoch 8/50
20/20 [==============================] - 0s 6ms/step - loss: 0.3967 - accuracy: 0.8257 - val_loss: 0.5007 - val_accuracy: 0.8052
Epoch 9/50
20/20 [==============================] - 0s 7ms/step - loss: 0.3720 - accuracy: 0.8339 - val_loss: 0.4876 - val_accuracy: 0.7922
Epoch 10/50
20/20 [==============================] - 0s 6ms/step - loss: 0.3603 - accuracy: 0.8485 - val_loss: 0.5746 - val_accuracy: 0.7857
Epoch 11/50
20/20 [==============================] - 0s 7ms/step - loss: 0.3579 - accuracy: 0.8436 - val_loss: 0.5300 - val_accuracy: 0.7987
Epoch 12/50
20/20 [==============================] - 0s 7ms/step - loss: 0.3243 - accuracy: 0.8664 - val_loss: 0.5532 - val_accuracy: 0.7922
Epoch 13/50
20/20

In [67]:
# Whole Model
def build_model(hp):
  model = Sequential()

  counter = 0
  for i in range(hp.Int('layers',1,10)):
    if counter == 0:
      model.add(Dense(
        hp.Int('units'+str(i),1,256),
        activation='relu',
        input_dim=8))
      model.add(Dropout(hp.Choice('drop'+str(i),[0.1,0.2,0.3,0.4,0.5])))
    else:
      model.add(Dense(
        hp.Int('units'+str(i),1,256),
        activation='relu'))
      model.add(Dropout(hp.Choice('drop'+str(i),[0.1,0.2,0.3,0.4,0.5])))
    counter+=1

  model.add(Dense(1,activation='sigmoid'))

  opt = hp.Choice('opti',['adam','rmsprop','sgd'])
  model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [68]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    directory='Final',
    project_name='ALL3'
)

In [69]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 23s


In [70]:
tuner.results_summary()

Results summary
Results in Final/ALL3
Showing 10 best trials
Trial summary
Hyperparameters:
layers: 7
units0: 216
drop0: 0.3
opti: adam
units1: 148
drop1: 0.3
units2: 52
drop2: 0.1
units3: 167
drop3: 0.5
units4: 109
drop4: 0.5
units5: 142
drop5: 0.5
units6: 118
drop6: 0.5
units7: 232
drop7: 0.4
units8: 135
drop8: 0.5
Score: 0.8051947951316833
Trial summary
Hyperparameters:
layers: 5
units0: 27
drop0: 0.3
opti: rmsprop
units1: 87
drop1: 0.4
units2: 9
drop2: 0.3
units3: 58
drop3: 0.2
units4: 106
drop4: 0.2
units5: 234
drop5: 0.1
units6: 239
drop6: 0.2
Score: 0.798701286315918
Trial summary
Hyperparameters:
layers: 5
units0: 233
drop0: 0.2
opti: rmsprop
units1: 237
drop1: 0.5
units2: 49
drop2: 0.5
units3: 113
drop3: 0.2
units4: 13
drop4: 0.2
units5: 193
drop5: 0.5
units6: 154
drop6: 0.2
Score: 0.798701286315918
Trial summary
Hyperparameters:
layers: 2
units0: 60
drop0: 0.2
opti: rmsprop
units1: 94
drop1: 0.2
units2: 236
drop2: 0.2
units3: 238
drop3: 0.3
units4: 18
drop4: 0.4
units5: 175
d

In [71]:
tuner.get_best_hyperparameters()[0].values

{'layers': 7,
 'units0': 216,
 'drop0': 0.3,
 'opti': 'adam',
 'units1': 148,
 'drop1': 0.3,
 'units2': 52,
 'drop2': 0.1,
 'units3': 167,
 'drop3': 0.5,
 'units4': 109,
 'drop4': 0.5,
 'units5': 142,
 'drop5': 0.5,
 'units6': 118,
 'drop6': 0.5,
 'units7': 232,
 'drop7': 0.4,
 'units8': 135,
 'drop8': 0.5}

In [72]:
model = tuner.get_best_models(num_models=1)[0]

In [73]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 216)               1944      
                                                                 
 dropout (Dropout)           (None, 216)               0         
                                                                 
 dense_1 (Dense)             (None, 148)               32116     
                                                                 
 dropout_1 (Dropout)         (None, 148)               0         
                                                                 
 dense_2 (Dense)             (None, 52)                7748      
                                                                 
 dropout_2 (Dropout)         (None, 52)                0         
                                                                 
 dense_3 (Dense)             (None, 167)               8

In [74]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),batch_size=32,initial_epoch=5)

Epoch 6/100
20/20 [==============================] - 1s 17ms/step - loss: 0.5188 - accuracy: 0.6792 - val_loss: 0.5535 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5088 - accuracy: 0.7296 - val_loss: 0.5456 - val_accuracy: 0.7662
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5081 - accuracy: 0.7557 - val_loss: 0.5253 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5016 - accuracy: 0.7476 - val_loss: 0.5312 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5012 - accuracy: 0.7508 - val_loss: 0.5183 - val_accuracy: 0.7727
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5245 - accuracy: 0.7638 - val_loss: 0.5277 - val_accuracy: 0.7532
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4848 - accuracy: 0.7622 - val_loss: 0.5010 - val_accuracy: 0.8117
Epoch 13/